In [ ]:
from astropy.coordinates import SkyCoord, EarthLocation
import astropy.coordinates as coord
import astropy.units as u
from tabulate import tabulate
from astropy.time import Time
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%load_ext autoreload
%autoreload 2
from Helper_funcs import *
from Scheduler import *

In [ ]:
connect = sqlite3.connect('Database.db')
add_randome_data(10)

In [ ]:
connect = sqlite3.connect('Database.db')
cont = sqlite_retrieve_table(connect, 'Schedule')
for i in cont:
    print(i)

In [ ]:
cont = sqlite_retrieve_table(connect, 'Observations')
for i in cont:
    print(i)

In [ ]:
from astroplan import Constraint
from scipy.interpolate import NearestNDInterpolator, interp1d
import numpy as np

class SeeingConstraint(Constraint): #TODO: Define this once figure out where we get seeing from
    """
    Constrains Seeing. Maybe use: https://www.meteoblue.com/en/weather/outdoorsports/seeing/groningen_netherlands_2755251

    Parameters
    ----------
    max : `~astropy.units.Quantity` or `None`
        Maximum altitude of the target (inclusive). `None` indicates no limit.
    boolean_constraint : bool
        If True, the constraint is treated as a boolean (True for within the
        limits and False for outside).  If False, the constraint returns a
        float on [0, 1], where 0 is the min altitude and 1 is the max.
    """

    def __init__(self, min=None, boolean_constraint=True):
        if max is None:
            self.max = 90*u.deg
        else:
            self.max = max

        self.boolean_constraint = boolean_constraint

    def compute_constraint(self, times, observer, targets):
        cached_altaz = _get_altaz(times, observer, targets)
        alt = cached_altaz['altaz'].alt
        if self.boolean_constraint:
            lowermask = self.min <= alt
            uppermask = alt <= self.max
            return lowermask & uppermask
        else:
            return max_best_rescale(alt, self.min, self.max)




# Trials of Scheduler

In [1]:
from Scheduler import Scheduling
file_path = r'.'
config = ''
sel=Scheduling(file_path, config)

wind10toast not installed
{'filter': 'None', 'obsID': 21}
{'filter': 'None', 'obsID': 23}
{'filter': 'H_Alpha', 'obsID': 25}
{'filter': 'H_Alpha', 'obsID': 5}


In [3]:
('obsID' in sel.scheduled_blocks[0].configuration.keys())#.configuration


dict_keys(['filter', 'obsID'])

# Narrow down datasets for interpolation function



In [ ]:
from Helper_funcs import load_all_csv
data = load_all_csv('./sky_bright')
csv_data = [[],[],[],[],[],[],[],[]]
for key in data:
    temp = data[key]
    temp[-1] = [float(i) for i in temp[-1]]
    for j in range(len(temp[-1])): #
        if temp[-1][j] < 1:
            break #J will keep the index of the last point
    for k in range(j+500,len(temp[-1])): #Starting point with conservative offset of 500 TODO: Find out how many datapoints are actually taken
        if temp[-1][k] > 1:
            break
    #We now have our start and stop indices
    #We will now append to a list to save the clipped data
    clipped = np.array(temp)[::,j:k]
    csv_data[0] += clipped[0].tolist() #Save datetime object as well
    csv_data[1] += [(i.date().days-i.date().replace(day=1,month=1)).days for i in clipped[0]]
    csv_data[2] += [int((i-i.replace(hour=0, minute=0,second=0)).total_seconds()/60) for i in clipped[0]]
    for i in range(1,len(csv_data)): #Skip dattime object 
        csv_data[i+1] += clipped[i]

In [ ]:
print(csv_data)